# Estimating Tour Mode Choice

This notebook illustrates how to re-estimate ActivitySim's auto ownership model.  The steps in the process are:
  - Run ActivitySim in estimation mode to read household travel survey files, run the ActivitySim submodels to write estimation data bundles (EDB) that contains the model utility specifications, coefficients, chooser data, and alternatives data for each submodel.
  - Read and transform the relevant EDB into the format required by the model estimation package [larch](https://larch.newman.me) and then re-estimate the model coefficients.  No changes to the model specification will be made.
  - Update the ActivitySim model coefficients and re-run the model in simulation mode.
  
The basic estimation workflow is shown below and explained in the next steps.

![estimation workflow](https://github.com/RSGInc/activitysim/raw/develop/docs/images/estimation_example.jpg)

# Load libraries

In [4]:
import larch  # !conda install larch #for estimation
import larch.util.activitysim
import pandas as pd
import numpy as np
import yaml 
import larch.util.excel
import os

# Required Inputs

In addition to a working ActivitySim model setup, estimation mode requires an ActivitySim format household travel survey.  An ActivitySim format household travel survey is very similar to ActivitySim's simulation model tables:

 - households
 - persons
 - tours
 - joint_tour_participants
 - trips (not yet implemented)

Examples of the ActivitySim format household travel survey are included in the [example_estimation data folders](https://github.com/RSGInc/activitysim/tree/develop/activitysim/examples/example_estimation).  The user is responsible for formatting their household travel survey into the appropriate format.  

After creating an ActivitySim format household travel survey, the `scripts/infer.py` script is run to append additional calculated fields.  An example of an additional calculated field is the `household:joint_tour_frequency`, which is calculated based on the `tours` and `joint_tour_participants` tables.  

The input survey files are below.

### Survey households

In [3]:
pd.read_csv("../data_sf/survey_data/override_persons.csv")

,person_id,household_id,age,PNUM,sex,pemploy,pstudent,ptype,school_taz,workplace_taz,free_parking_at_work,cdap_activity,mandatory_tour_frequency,_escort,_shopping,_othmaint,_othdiscr,_eatout,_social,non_mandatory_tour_frequency
0,166,166,54,1,2,3,3,4,-1,-1,False,N,NaN,0,0,0,0,1,0,4
1,197,197,46,1,2,3,3,4,-1,-1,False,N,NaN,0,1,0,0,0,0,16
2,268,268,46,1,1,3,3,4,-1,-1,False,N,NaN,0,0,1,1,0,0,9
3,375,375,54,1,2,3,3,4,-1,-1,False,N,NaN,0,0,1,0,0,0,8
4,387,387,44,1,2,3,3,4,-1,-1,False,N,NaN,1,0,0,1,0,0,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4401,7554799,2863464,93,1,2,3,3,5,-1,-1,False,N,NaN,0,0,0,1,0,0,1
4402,7554818,2863483,68,1,1,3,3,5,-1,-1,False,N,NaN,0,0,1,1,0,0,9
4403,7555141,2863806,93,1,2,3,3,5,-1,-1,False,N,NaN,0,2,0,1,0,0,17
4404,7555853,2864518,71,1,1,3,3,5,-1,-1,False,N,NaN,0,0,0,0,0,1,2


### Survey persons

In [4]:
pd.read_csv("../data_sf/survey_data/override_persons.csv")

,person_id,household_id,age,PNUM,sex,pemploy,pstudent,ptype,school_taz,workplace_taz,free_parking_at_work,cdap_activity,mandatory_tour_frequency,_escort,_shopping,_othmaint,_othdiscr,_eatout,_social,non_mandatory_tour_frequency
0,166,166,54,1,2,3,3,4,-1,-1,False,N,NaN,0,0,0,0,1,0,4
1,197,197,46,1,2,3,3,4,-1,-1,False,N,NaN,0,1,0,0,0,0,16
2,268,268,46,1,1,3,3,4,-1,-1,False,N,NaN,0,0,1,1,0,0,9
3,375,375,54,1,2,3,3,4,-1,-1,False,N,NaN,0,0,1,0,0,0,8
4,387,387,44,1,2,3,3,4,-1,-1,False,N,NaN,1,0,0,1,0,0,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4401,7554799,2863464,93,1,2,3,3,5,-1,-1,False,N,NaN,0,0,0,1,0,0,1
4402,7554818,2863483,68,1,1,3,3,5,-1,-1,False,N,NaN,0,0,1,1,0,0,9
4403,7555141,2863806,93,1,2,3,3,5,-1,-1,False,N,NaN,0,2,0,1,0,0,17
4404,7555853,2864518,71,1,1,3,3,5,-1,-1,False,N,NaN,0,0,0,0,0,1,2


### Survey tours

In [5]:
pd.read_csv("../data_sf/survey_data/override_tours.csv")

,tour_id,survey_tour_id,person_id,household_id,tour_type,tour_category,destination,origin,start,end,tour_mode,survey_parent_tour_id,parent_tour_id,composition,tdd,atwork_subtour_frequency
0,25820,258200,629,629,school,mandatory,133.0,131.0,12.0,15.0,WALK,NaN,NaN,NaN,115,NaN
1,52265,522650,1274,1274,school,mandatory,188.0,166.0,9.0,15.0,WALK_LOC,NaN,NaN,NaN,76,NaN
2,1117937,11179370,27266,27266,school,mandatory,133.0,9.0,17.0,18.0,WALK_HVY,NaN,NaN,NaN,163,NaN
3,1148523,11485230,28012,28012,school,mandatory,12.0,10.0,17.0,22.0,WALK_LRF,NaN,NaN,NaN,167,NaN
4,1208547,12085470,29476,29476,school,mandatory,13.0,16.0,8.0,15.0,WALK_LOC,NaN,NaN,NaN,61,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5768,302942627,3029426270,7388844,2750003,maint,atwork,5.0,7.0,14.0,14.0,WALK,3.029426e+09,302942643.0,NaN,135,NaN
5769,305120465,3051204650,7441962,2758909,maint,atwork,110.0,2.0,12.0,13.0,SHARED2FREE,3.051205e+09,305120481.0,NaN,113,NaN
5770,308000655,3080006550,7512211,2820876,eat,atwork,14.0,1.0,12.0,13.0,WALK,3.080007e+09,308000690.0,NaN,113,NaN
5771,308073840,3080738400,7513996,2822661,eat,atwork,69.0,107.0,8.0,16.0,SHARED3FREE,3.080739e+09,308073875.0,NaN,62,NaN


### Survey joint tour participants

In [6]:
pd.read_csv("../data_sf/survey_data/survey_joint_tour_participants.csv")

,participant_id,tour_id,household_id,person_id,participant_num
0,22095828301,220958283,2223759,5389226,1
1,22095828302,220958283,2223759,5389227,2
2,14429508701,144295087,1606646,3519392,1
3,14429508702,144295087,1606646,3519393,2
4,28367651801,283676518,2628704,6918939,1
...,...,...,...,...,...
226,16297928102,162979281,1769918,3975105,2
227,16297928103,162979281,1769918,3975106,3
228,16297928104,162979281,1769918,3975107,4
229,26353054902,263530549,2519358,6427575,1


# Example Setup if Needed

To avoid duplication of inputs, especially model settings and expressions, the `example_estimation` depends on the `example`.  The following commands create an example setup for use.  The location of these example setups (i.e. the folders) are important because the paths are referenced in this notebook.  The commands below download the skims.omx for the SF county example from the [activitysim resources repository](https://github.com/RSGInc/activitysim_resources).

In [7]:
!activitysim create -e example_estimation_sf -d test

copying configs ...
copying configs ...
copying data_sf ...
downloading skims.omx ...
copying output ...
copied! new project files are in C:\projects\development\activitysim_rsg\activitysim\examples\example_estimation\notebooks\test\example_estimation_sf
the copied example can be run with

  activitysim run -w test\example_estimation_sf


# Run the Estimation Example

The next step is to run the model with an `estimation.yaml` settings file with the following settings in order to output the EDB for all submodels:

```
enable=True

bundles:
  - school_location
  - workplace_location
  - auto_ownership
  - free_parking
  - cdap
  - mandatory_tour_frequency
  - mandatory_tour_scheduling
  - joint_tour_frequency
  - joint_tour_composition
  - joint_tour_participation
  - joint_tour_destination
  - joint_tour_scheduling
  - non_mandatory_tour_frequency
  - non_mandatory_tour_destination
  - non_mandatory_tour_scheduling
  - tour_mode_choice
  - atwork_subtour_frequency
  - atwork_subtour_destination
  - atwork_subtour_scheduling
  - atwork_subtour_mode_choice

survey_tables:
  households:
    file_name: survey_data/override_households.csv
    index_col: household_id
  persons:
    file_name:  survey_data/override_persons.csv
    index_col: person_id
  tours:
    file_name:  survey_data/override_tours.csv
  joint_tour_participants:
    file_name:  survey_data/override_joint_tour_participants.csv
```

This enables the estimation mode functionality, identifies which models to run and their output estimation data bundles (EDBs), and the input survey tables, which include the override settings for each model choice.  

With this setup, the model will output an EBD with the following tables for this submodel:
  - model settings - tour_mode_choice_model_settings.yaml
  - coefficients - tour_mode_choice_coefficients.csv
  - coefficients template by tour purpose - tour_mode_choice_coefficients_template.csv
  - utilities specification - tour_mode_choice_SPEC.csv
  - chooser data - tour_mode_choice_values_combined.csv
  
The following code runs the software in estimation mode, inheriting the settings from the simulation setup and using the San Francisco county data setup.  It produces the EDB for all submodels but runs all the model steps identified in the inherited settings file.  

In [8]:
%cd test
!activitysim run -c configs_estimation/configs -c configs -o output -d data_sf

C:\projects\development\activitysim_rsg\activitysim\examples\example_estimation\notebooks\test
Configured logging using basicConfig
INFO:activitysim:Configured logging using basicConfig
INFO:activitysim.cli.run:using configs_dir: ['configs_estimation/configs', 'configs']
INFO:activitysim.cli.run:using data_dir: ['data_sf']
INFO:activitysim.cli.run:using output_dir: ['output']
INFO - Read logging configuration from: configs_estimation/configs\logging.yaml
INFO - setting households_sample_size: 0
INFO - setting chunk_size: 0
INFO - setting multiprocess: None
INFO - setting num_processes: None
INFO - setting resume_after: None
INFO - run single process simulation
INFO - open_pipeline
INFO - Set random seed base to 0
INFO - Time to execute open_pipeline : 0.022 seconds (0.0 minutes)
INFO - preload_injectables
INFO - Time to execute preload_injectables : 0.0 seconds (0.0 minutes)
INFO - Reading CSV file data_sf\land_use.csv
INFO - renaming columns: {'ZONE': 'TAZ', 'COUNTY': 'county_id'}
INF

INFO - cdap: end estimation
INFO - cdap_activity top 10 value counts:
M    2623
N    1211
H     572
Name: cdap_activity, dtype: int64
INFO - cdap crosstabs:
cdap_activity    H     M     N   All
ptype                               
1              154  1609   158  1921
2               47   331   118   496
3               33   230    66   329
4              114     0   420   534
5              117     0   354   471
6               22    42     8    72
7               38   274    40   352
8               47   137    47   231
All            572  2623  1211  4406
INFO - Running mandatory_tour_frequency with 2623 persons
INFO - Initialize Estimator for'mandatory_tour_frequency'
DEBUG - mandatory_tour_frequency: estimate.write_spec: output\estimation_data_bundle\mandatory_tour_frequency\mandatory_tour_frequency_SPEC.csv
DEBUG - mandatory_tour_frequency: write_table write: coefficients
DEBUG - mandatory_tour_frequency: write_table cache: choosers
INFO - Running chunk 1 of 1 size 2623
DEBUG - ma

# Read EDB

The next step is to read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

In [5]:
edb_directory = "output/estimation_data_bundle/tour_mode_choice/"

def read_csv(filename, **kwargs):
    return pd.read_csv(os.path.join(edb_directory, filename), **kwargs)

In [6]:
coefficients = read_csv(
    "tour_mode_choice_coefficients.csv",
    index_col='coefficient_name',
)
coef_template = read_csv(
    "tour_mode_choice_coefficients_template.csv", 
    index_col='coefficient_name',
)
spec = read_csv("tour_mode_choice_SPEC.csv")
values = read_csv("tour_mode_choice_values_combined.csv")

### Model settings

In [7]:
settings = yaml.load( 
    open(os.path.join(edb_directory, "tour_mode_choice_model_settings.yaml"),"r"), 
    Loader=yaml.SafeLoader,
)

settings

{'LOGIT_TYPE': 'NL',
 'NESTS': {'name': 'root',
  'coefficient': 'coef_nest_root',
  'alternatives': [{'name': 'AUTO',
    'coefficient': 'coef_nest_AUTO',
    'alternatives': [{'name': 'DRIVEALONE',
      'coefficient': 'coef_nest_AUTO_DRIVEALONE',
      'alternatives': ['DRIVEALONEFREE', 'DRIVEALONEPAY']},
     {'name': 'SHAREDRIDE2',
      'coefficient': 'coef_nest_AUTO_SHAREDRIDE2',
      'alternatives': ['SHARED2FREE', 'SHARED2PAY']},
     {'name': 'SHAREDRIDE3',
      'coefficient': 'coef_nest_AUTO_SHAREDRIDE3',
      'alternatives': ['SHARED3FREE', 'SHARED3PAY']}]},
   {'name': 'NONMOTORIZED',
    'coefficient': 'coef_nest_NONMOTORIZED',
    'alternatives': ['WALK', 'BIKE']},
   {'name': 'TRANSIT',
    'coefficient': 'coef_nest_TRANSIT',
    'alternatives': [{'name': 'WALKACCESS',
      'coefficient': 'coef_nest_TRANSIT_WALKACCESS',
      'alternatives': ['WALK_LOC',
       'WALK_LRF',
       'WALK_EXP',
       'WALK_HVY',
       'WALK_COM']},
     {'name': 'DRIVEACCESS',
      

### Coefficients

In [8]:
coefficients

,value,constrain
coefficient_name,,
coef_nest_root,1.000,T
coef_nest_AUTO,0.720,T
coef_nest_AUTO_DRIVEALONE,0.350,T
coef_nest_AUTO_SHAREDRIDE2,0.350,T
coef_nest_AUTO_SHAREDRIDE3,0.350,T
...,...,...
walk_transit_CBD_ASC_atwork,0.564,F
drive_transit_CBD_ASC_eatout_escort_othdiscr_othmaint_shopping_social,0.525,F
drive_transit_CBD_ASC_school_univ,0.672,F


### Coef_template - coefficients by tour purpose

In [9]:
coef_template

,eatout,escort,othdiscr,othmaint,school,shopping,social,univ,work,atwork
coefficient_name,,,,,,,,,,
coef_nest_root,coef_nest_root,coef_nest_root,coef_nest_root,coef_nest_root,coef_nest_root,coef_nest_root,coef_nest_root,coef_nest_root,coef_nest_root,coef_nest_root
coef_nest_AUTO,coef_nest_AUTO,coef_nest_AUTO,coef_nest_AUTO,coef_nest_AUTO,coef_nest_AUTO,coef_nest_AUTO,coef_nest_AUTO,coef_nest_AUTO,coef_nest_AUTO,coef_nest_AUTO
coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE,coef_nest_AUTO_DRIVEALONE
coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2,coef_nest_AUTO_SHAREDRIDE2
coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3,coef_nest_AUTO_SHAREDRIDE3
...,...,...,...,...,...,...,...,...,...,...
express_bus_ASC,express_bus_ASC_eatout_escort_othdiscr_othmain...,express_bus_ASC_eatout_escort_othdiscr_othmain...,express_bus_ASC_eatout_escort_othdiscr_othmain...,express_bus_ASC_eatout_escort_othdiscr_othmain...,express_bus_ASC_school_univ,express_bus_ASC_eatout_escort_othdiscr_othmain...,express_bus_ASC_eatout_escort_othdiscr_othmain...,express_bus_ASC_school_univ,express_bus_ASC_work,express_bus_ASC_eatout_escort_othdiscr_othmain...
heavy_rail_ASC,heavy_rail_ASC_eatout_escort_othdiscr_othmaint...,heavy_rail_ASC_eatout_escort_othdiscr_othmaint...,heavy_rail_ASC_eatout_escort_othdiscr_othmaint...,heavy_rail_ASC_eatout_escort_othdiscr_othmaint...,heavy_rail_ASC_school_univ,heavy_rail_ASC_eatout_escort_othdiscr_othmaint...,heavy_rail_ASC_eatout_escort_othdiscr_othmaint...,heavy_rail_ASC_school_univ,heavy_rail_ASC_work,heavy_rail_ASC_eatout_escort_othdiscr_othmaint...
commuter_rail_ASC,commuter_rail_ASC_eatout_escort_othdiscr_othma...,commuter_rail_ASC_eatout_escort_othdiscr_othma...,commuter_rail_ASC_eatout_escort_othdiscr_othma...,commuter_rail_ASC_eatout_escort_othdiscr_othma...,commuter_rail_ASC_school_univ,commuter_rail_ASC_eatout_escort_othdiscr_othma...,commuter_rail_ASC_eatout_escort_othdiscr_othma...,commuter_rail_ASC_school_univ,commuter_rail_ASC_work,commuter_rail_ASC_eatout_escort_othdiscr_othma...


### Utility specifications

In [12]:
# Remove apostrophes from Label names
spec['Label'] = spec['Label'].str.replace("'","")
spec

,Label,Description,Expression,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,...,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED
0,#,Drive alone no toll,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,util_DRIVEALONEFREE_Unavailable,DRIVEALONEFREE - Unavailable,sov_available == False,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,util_DRIVEALONEFREE_Unavailable_for_zero_auto_...,DRIVEALONEFREE - Unavailable for zero auto hou...,auto_ownership == 0,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,util_DRIVEALONEFREE_Unavailable_for_persons_le...,DRIVEALONEFREE - Unavailable for persons less ...,age < 16,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,util_DRIVEALONEFREE_Unavailable_for_joint_tours,DRIVEALONEFREE - Unavailable for joint tours,is_joint == True,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,#,FIXME - skims aren't symmetrical,so we have to make sure they can get back,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
341,util_Walk_not_available_for_long_distances,Walk not available for long distances,@od_skims.max('DISTWALK') > 3,NaN,NaN,NaN,NaN,NaN,NaN,-999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
342,util_Bike_not_available_for_long_distances,Bike not available for long distances,@od_skims.max('DISTBIKE') > 8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343,util_Drive_alone_not_available_for_escort_tours,Drive alone not available for escort tours,is_escort,-999,-999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Check for double-parameters
ss = spec.query("Label!='#'").iloc[:,3:].stack().str.split("*")
st = ss.apply(lambda x: len(x))>1
assert len(ss[st]) == 0

### Chooser and alternative values

In [14]:
# Remove apostrophes from column names
values.columns = values.columns.str.replace("'","")
values.fillna(0, inplace=True)
values

,tour_id,model_choice,override_choice,util_DRIVEALONEFREE_Unavailable,util_DRIVEALONEFREE_Unavailable_for_zero_auto_households,util_DRIVEALONEFREE_Unavailable_for_persons_less_than_16,util_DRIVEALONEFREE_Unavailable_for_joint_tours,util_DRIVEALONEFREE_Unavailable_if_didnt_drive_to_work,util_DRIVEALONEFREE_In_vehicle_time,util_DRIVEALONEFREE_Terminal_time,...,walk_heavyrail_available,walk_lrf_available,walk_ferry_available,drive_local_available,drive_commuter_available,drive_express_available,drive_heavyrail_available,drive_lrf_available,drive_ferry_available,destination_in_cbd
0,6812,WALK,WALK,0.0,1.0,0.0,0.0,0.0,1.620000,12.85052,...,False,False,False,False,False,False,False,False,False,0
1,8110,WALK_LRF,WALK_LRF,0.0,1.0,0.0,0.0,0.0,13.010000,22.05276,...,False,True,False,False,False,False,False,False,False,1
2,11013,DRIVEALONEFREE,DRIVEALONEFREE,0.0,0.0,0.0,0.0,0.0,25.420000,19.56452,...,False,True,False,True,True,False,True,False,False,1
3,11016,DRIVEALONEFREE,DRIVEALONEFREE,0.0,0.0,0.0,0.0,0.0,18.779999,14.01624,...,False,True,False,True,True,False,True,False,False,0
4,15403,DRIVEALONEFREE,SHARED2FREE,0.0,0.0,0.0,0.0,0.0,7.180000,19.88140,...,True,False,False,True,True,False,True,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5318,309760814,SHARED2FREE,SHARED2FREE,0.0,0.0,0.0,0.0,0.0,13.700001,16.73920,...,False,False,False,True,True,False,True,False,False,1
5319,309760815,DRIVEALONEFREE,DRIVEALONEFREE,0.0,0.0,0.0,0.0,0.0,4.770000,18.44424,...,False,False,False,True,True,False,True,False,False,0
5320,309790009,BIKE,BIKE,0.0,0.0,0.0,0.0,0.0,14.809999,21.41740,...,False,False,False,True,True,False,True,False,False,1
5321,309796968,SHARED2FREE,SHARED2FREE,0.0,0.0,0.0,0.0,0.0,12.880000,10.61908,...,False,True,False,True,True,False,True,False,False,0


# Data Processing and Estimation Setup

The next step is to transform the EDB for larch for model re-estimation.  

In [15]:
from larch import P,X

### Alternatives

In [16]:
alt_names = list(spec.columns[3:])
alt_codes = np.arange(1,len(alt_names)+1)
alt_names_to_codes = dict(zip(alt_names, alt_codes))
alt_codes_to_names = dict(zip(alt_codes, alt_names))
alt_names_to_codes

{'DRIVEALONEFREE': 1,
 'DRIVEALONEPAY': 2,
 'SHARED2FREE': 3,
 'SHARED2PAY': 4,
 'SHARED3FREE': 5,
 'SHARED3PAY': 6,
 'WALK': 7,
 'BIKE': 8,
 'WALK_LOC': 9,
 'WALK_LRF': 10,
 'WALK_EXP': 11,
 'WALK_HVY': 12,
 'WALK_COM': 13,
 'DRIVE_LOC': 14,
 'DRIVE_LRF': 15,
 'DRIVE_EXP': 16,
 'DRIVE_HVY': 17,
 'DRIVE_COM': 18,
 'TAXI': 19,
 'TNC_SINGLE': 20,
 'TNC_SHARED': 21}

### Remove choosers with invalid observed choice

In [29]:
values = values[values.override_choice.isin(alt_names)]

### Nesting structure

In [18]:
tree = larch.util.activitysim.construct_nesting_tree(alt_names, settings['NESTS'])
tree

In [19]:
tree.elemental_names()

{1: 'DRIVEALONEFREE',
 2: 'DRIVEALONEPAY',
 3: 'SHARED2FREE',
 4: 'SHARED2PAY',
 5: 'SHARED3FREE',
 6: 'SHARED3PAY',
 7: 'WALK',
 8: 'BIKE',
 9: 'WALK_LOC',
 10: 'WALK_LRF',
 11: 'WALK_EXP',
 12: 'WALK_HVY',
 13: 'WALK_COM',
 14: 'DRIVE_LOC',
 15: 'DRIVE_LRF',
 16: 'DRIVE_EXP',
 17: 'DRIVE_HVY',
 18: 'DRIVE_COM',
 19: 'TAXI',
 20: 'TNC_SINGLE',
 21: 'TNC_SHARED'}

### List tour purposes

In [20]:
purposes = list(coef_template.columns)
purposes

['eatout',
 'escort',
 'othdiscr',
 'othmaint',
 'school',
 'shopping',
 'social',
 'univ',
 'work',
 'atwork']

### Setup purpose specific models

In [30]:
m = {purpose:larch.Model(graph=tree) for purpose in purposes}

In [31]:
for alt_code, alt_name in tree.elemental_names().items():
    # Read in base utility function for this alt_name
    u = larch.util.activitysim.linear_utility_from_spec(
        spec, x_col='Label', p_col=alt_name, 
        ignore_x=('#',), 
    )
    for purpose in purposes:
        # Modify utility function based on template for purpose
        u_purp = sum(
            (
                P(coef_template[purpose].get(i.param,i.param)) 
                * i.data * i.scale
            )
            for i in u
        )
        m[purpose].utility_co[alt_code] = u_purp


### Set parameter values

In [32]:
for model in m.values():
    larch.util.activitysim.explicit_value_parameters(model)

In [33]:
larch.util.activitysim.apply_coefficients(coefficients, m)

### Survey choice

In [38]:
values['override_choice_code'] = values.override_choice.map(alt_names_to_codes)

### Availability

In [41]:
av = True # all alternatives are available

In [42]:
d = larch.DataFrames(
    co=values.set_index('tour_id'),
    av=av,
    alt_codes=alt_codes,
    alt_names=alt_names,
)

In [43]:
for purpose, model in m.items():
    model.dataservice = d.selector_co(f"tour_type=='{purpose}'")
    model.choice_co_code = 'override_choice_code'

In [37]:
from larch.model.model_group import ModelGroup
mg = ModelGroup(m.values())

# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has two built-in estimation methods: BHHH and SLSQP.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.  SLSQP is safer, but slower, and may need additional iterations.

In [46]:
mg.estimate(method='SLSQP', options={'maxiter':1000})
#mg.estimate(method='BHHH', options={'maxiter':1000})

req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided


,value,initvalue,nullvalue,minimum,maximum,holdfast,note,std_err,t_stat,robust_std_err,robust_t_stat,likelihood_ratio
-999,-999.000000,-999.0,-999.0,-999.0,-999.0,1,,0.000000,NaN,0.0,NaN,NaN
1,1.000000,1.0,1.0,1.0,1.0,1,,0.000000,NaN,0.0,NaN,NaN
bike_ASC_auto_deficient_eatout,-1.569111,0.0,0.0,NaN,NaN,0,,1.221903,-1.284153,0.0,-inf,NaN
bike_ASC_auto_sufficient_eatout,-1.200347,0.0,0.0,NaN,NaN,0,,0.464989,-2.581453,0.0,-inf,NaN
bike_ASC_no_auto_eatout,0.868071,0.0,0.0,NaN,NaN,0,,7.078790,0.122630,0.0,inf,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
walk_ASC_no_auto_atwork,6.669213,0.0,0.0,NaN,NaN,0,,0.000000,inf,0.0,inf,NaN
walk_transit_ASC_auto_deficient_atwork,-2.998829,0.0,0.0,NaN,NaN,0,,0.000000,-inf,0.0,-inf,NaN
walk_transit_ASC_auto_sufficient_atwork,-3.401027,0.0,0.0,NaN,NaN,0,,0.000000,-inf,0.0,-inf,NaN
walk_transit_ASC_no_auto_atwork,2.704188,0.0,0.0,NaN,NaN,0,,0.000000,inf,0.0,inf,NaN


c:\programdata\anaconda3\envs\asimtest2\lib\site-packages\numpy\core\_methods.py:38: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
c:\programdata\anaconda3\envs\asimtest2\lib\site-packages\ipykernel_launcher.py:1: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  """Entry point for launching an IPython kernel.
c:\programdata\anaconda3\envs\asimtest2\lib\site-packages\larch\linalg\__init__.py:18: UserWarning: minimum eig 0.0 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")


┣                x: -999                                      -999.000000
┃                   1                                            1.000000
┃                   bike_ASC_auto_deficient_eatout              -1.569111
┃                   bike_ASC_auto_sufficient_eatout             -1.200347
┃                   bike_ASC_no_auto_eatout                      0.868071
┃                                                                 ...    
┃                   walk_ASC_no_auto_atwork                      6.669213
┃                   walk_transit_ASC_auto_deficient_atwork      -2.998829
┃                   walk_transit_ASC_auto_sufficient_atwork     -3.401027
┃                   walk_transit_ASC_no_auto_atwork              2.704188
┃                   walk_transit_CBD_ASC_atwork                  0.564000
┃                   Length: 301, dtype: float64
┣          loglike: -inf
┣        d_loglike: -999                                       0.000000
┃                   1                                          0.000000
┃                   bike_ASC_auto_deficient_eatout             0.764932
┃                   bike_ASC_auto_sufficient_eatout           -2.709202
┃                   bike_ASC_no_auto_eatout                   -1.386687
┃                                                                ...   
┃                   walk_ASC_no_auto_atwork                    0.000000
┃                   walk_transit_ASC_auto_deficient_atwork     0.000000
┃                   walk_transit_ASC_auto_sufficient_atwork    0.000000
┃                   walk_transit_ASC_no_auto_atwork            0.000000
┃                   walk_transit_CBD_ASC_atwork                0.000000
┃                   Length: 301, dtype: float64
┣              nit: 1
┣             nfev: 12
┣             njev: 1
┣           status: 0
┣          message: 'Optimization terminated successfully.'
┣          success: True
┣     elapsed_time: datetime.timedelta(seconds=7, microseconds=78056)
┣           method: 'SLSQP'
┣          n_cases: 5323
┣ iteration_number: 1
┣          logloss: inf

### Estimated coefficients

In [49]:
mg.parameter_summary()

,Value,Std Err,t Stat,Signif,Like Ratio,Null Value
-999,-999.,0.00,NA,,NA,-999.00
1,1.00,0.00,NA,,NA,1.00
bike_ASC_auto_deficient_eatout,-1.57,1.22,-1.28,,NA,0.00
bike_ASC_auto_sufficient_eatout,-1.20,0.465,-2.58,**,NA,0.00
bike_ASC_no_auto_eatout,0.868,150.,0.01,,NA,0.00
coef_age010_trn_multiplier_eatout_escort_othdiscr_othmaint_shopping_social_work,3.16e-11,0.415,0.00,,NA,0.00
coef_age1619_da_multiplier_eatout_escort_othdiscr_othmaint_shopping_social_work,-7.26e-10,0.259,-0.00,,NA,0.00
coef_age16p_sr_multiplier_eatout_escort_othdiscr_othmaint_shopping_social,-1.37,0.250,-5.47,***,NA,0.00
coef_hhsize1_sr_multiplier_eatout_escort_othdiscr_othmaint_school_shopping_social_univ_atwork,2.10e-09,0.112,0.00,,NA,0.00
coef_hhsize2_sr_multiplier_eatout_escort_othdiscr_othmaint_shopping_social_work_atwork,-1.47e-08,0.0824,-0.00,,NA,0.00


# Output Estimation Results

In [50]:
est_names = [j for j in coefficients.index if j in mg.pf.index]
coefficients.loc[est_names, 'value'] = mg.pf.loc[est_names, 'value']

In [51]:
os.makedirs(os.path.join(edb_directory,'estimated'), exist_ok=True)

### Write the re-estimated coefficients file

In [52]:
coefficients.reset_index().to_csv(
    os.path.join(
        edb_directory, 
        'estimated',
        "tour_mode_choice_coefficients_revised.csv",
    ),
    index=False,
)

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [53]:
for purpose, model in m.items():
    model.to_xlsx(
        os.path.join(
            edb_directory, 
            'estimated',
            f"tour_mode_choice_{purpose}_model_estimation.xlsx",
        ), data_statistics=False
    )

# Next Steps

The final step is to either manually or automatically copy the `tour_mode_choice_coefficients_revised.csv` file to the configs folder, rename it to `coefficients.csv`, and run ActivitySim in simulation mode.

In [54]:
pd.read_csv(os.path.join(edb_directory,'estimated',"tour_mode_choice_coefficients_revised.csv"))

,coefficient_name,value,constrain
0,coef_nest_root,1.000,T
1,coef_nest_AUTO,0.720,T
2,coef_nest_AUTO_DRIVEALONE,0.350,T
3,coef_nest_AUTO_SHAREDRIDE2,0.350,T
4,coef_nest_AUTO_SHAREDRIDE3,0.350,T
...,...,...,...
301,walk_transit_CBD_ASC_atwork,0.564,F
302,drive_transit_CBD_ASC_eatout_escort_othdiscr_o...,0.525,F
303,drive_transit_CBD_ASC_school_univ,0.672,F
304,drive_transit_CBD_ASC_work,1.100,F
